In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/nfl-big-data-bowl-2020/train.csv
/kaggle/input/nfl-big-data-bowl-2020/kaggle/competitions/nflrush/test.csv.encrypted
/kaggle/input/nfl-big-data-bowl-2020/kaggle/competitions/nflrush/competition.cpython-36m-x86_64-linux-gnu.so
/kaggle/input/nfl-big-data-bowl-2020/kaggle/competitions/nflrush/__init__.py
/kaggle/input/nfl-big-data-bowl-2020/kaggle/competitions/nflrush/sample_submission.csv.encrypted


In [2]:
import os
TRAIN_ABLE_FALSE=True
if TRAIN_ABLE_FALSE:
    os.environ['CUDA_VISIBLE_DEVICES'] = "1"
import numpy as np
import pandas as pd
import sklearn.metrics as mtr
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures

from keras.layers import Dense,Input,Flatten,concatenate,Dropout,Lambda,BatchNormalization,LeakyReLU,PReLU,ELU,ThresholdedReLU,Concatenate
from keras.models import Model
import keras.backend as K
from keras.callbacks import Callback
from  keras.callbacks import EarlyStopping,ModelCheckpoint
import datetime
import warnings
warnings.filterwarnings('ignore')

TRAIN_OFFLINE = False


pd.set_option('display.max_columns', 150)
pd.set_option('display.max_rows', 150)

Using TensorFlow backend.


In [3]:
train=pd.read_csv("/kaggle/input/nfl-big-data-bowl-2020/train.csv")

In [4]:
outcomes = train[['GameId','PlayId','Yards']].drop_duplicates()

In [5]:
train.head()

,GameId,PlayId,Team,X,Y,S,A,Dis,Orientation,Dir,NflId,DisplayName,JerseyNumber,Season,YardLine,Quarter,GameClock,PossessionTeam,Down,Distance,FieldPosition,HomeScoreBeforePlay,VisitorScoreBeforePlay,NflIdRusher,OffenseFormation,OffensePersonnel,DefendersInTheBox,DefensePersonnel,PlayDirection,TimeHandoff,TimeSnap,Yards,PlayerHeight,PlayerWeight,PlayerBirthDate,PlayerCollegeName,Position,HomeTeamAbbr,VisitorTeamAbbr,Week,Stadium,Location,StadiumType,Turf,GameWeather,Temperature,Humidity,WindSpeed,WindDirection
0,2017090700,20170907000118,away,73.91,34.84,1.69,1.13,0.40,81.99,177.18,496723,Eric Berry,29,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-0,212,12/29/1988,Tennessee,SS,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
1,2017090700,20170907000118,away,74.67,32.64,0.42,1.35,0.01,27.61,198.70,2495116,Allen Bailey,97,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-3,288,03/25/1989,Miami,DE,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
2,2017090700,20170907000118,away,74.00,33.20,1.22,0.59,0.31,3.01,202.73,2495493,Justin Houston,50,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-3,270,01/21/1989,Georgia,DE,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
3,2017090700,20170907000118,away,71.46,27.70,0.42,0.54,0.02,359.77,105.64,2506353,Derrick Johnson,56,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-3,245,11/22/1982,Texas,ILB,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
4,2017090700,20170907000118,away,69.32,35.42,1.82,2.43,0.16,12.63,164.31,2530794,Ron Parker,38,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-0,206,08/17/1987,Newberry,FS,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW


In [6]:
#转化为秒钟
def strtoseconds(txt):
    txt = txt.split(':')
    ans = int(txt[0])*60 + int(txt[1]) + int(txt[2])/60
    return ans

#转化为浮点数
def strtofloat(x):
    try:
        return float(x)
    except:
        return -1

#天气转化为数字表示，数字越大代表天气越好，越小代表越坏，如下雨
def map_weather(txt):
    ans = 1
    if pd.isna(txt):
        return 0
    if 'partly' in txt:
        ans*=0.5
    if 'climate controlled' in txt or 'indoor' in txt:
        return ans*3
    if 'sunny' in txt or 'sun' in txt:
        return ans*2
    if 'clear' in txt:
        return ans
    if 'cloudy' in txt:
        return -ans
    if 'rain' in txt or 'rainy' in txt:
        return -2*ans
    if 'snow' in txt:
        return -3*ans
    return 0

#进攻队位置分组，例如：1 OL表示一个1攻击线峰，2QB表示两个主帅
def OffensePersonnelSplit(x):
    dic = {'DB' : 0, 'DL' : 0, 'LB' : 0, 'OL' : 0, 'QB' : 0, 'RB' : 0, 'TE' : 0, 'WR' : 0}
    for xx in x.split(","):
        xxs = xx.split(" ")
        dic[xxs[-1]] = int(xxs[-2])
    return dic

#防卫队位置分组，例如：1 OL表示一个1攻击线峰，2QB表示两个主帅
def DefensePersonnelSplit(x):
    dic = {'DB' : 0, 'DL' : 0, 'LB' : 0, 'OL' : 0}
    for xx in x.split(","):
        xxs = xx.split(" ")
        dic[xxs[-1]] = int(xxs[-2])
    return dic

#方向转换，转为0-359
def orientation_to_cat(x):
    x = np.clip(x, 0, 360 - 1)
    try:
        return str(int(x/15))
    except:
        return "nan"

#定义对train进行预处理函数
def preprocess(train):
    ## GameClock
    #将GameClock转化为秒
    train['GameClock_sec'] = train['GameClock'].apply(strtoseconds)
    #记录GameClock的分钟
    train["GameClock_minute"] = train["GameClock"].apply(lambda x : x.split(":")[0]).astype("object")
    
    ## Height,将尺转化为数字计算的Height
    train['PlayerHeight_dense'] = train['PlayerHeight'].apply(lambda x: 12*int(x.split('-')[0])+int(x.split('-')[1]))
    
    ## Time,将时间规范化到2018-11-01 00:44:12这种形式
    train['TimeHandoff'] = train['TimeHandoff'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"))
    train['TimeSnap'] = train['TimeSnap'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"))
    train['TimeDelta'] = train.apply(lambda row: (row['TimeHandoff'] - row['TimeSnap']).total_seconds(), axis=1)
    train['PlayerBirthDate'] = train['PlayerBirthDate'].apply(lambda x: datetime.datetime.strptime(x, "%m/%d/%Y"))
    
    ## Age,获取Player的年龄
    seconds_in_year = 60*60*24*365.25
    train['PlayerAge'] = train.apply(lambda row: (row['TimeHandoff']-row['PlayerBirthDate']).total_seconds()/seconds_in_year, axis=1)
    train["PlayerAge_ob"] = train['PlayerAge'].astype(np.int).astype("object")
    
    ## WindSpeed
    train['WindSpeed_ob'] = train['WindSpeed'].apply(lambda x: x.str.lower().replace('mph', '').strip() if not pd.isna(x) else x) #replace函数:.replace(low,new)
    train['WindSpeed_ob'] = train['WindSpeed_ob'].apply(lambda x: (int(x.split('-')[0])+int(x.split('-')[1]))/2 if not pd.isna(x) and '-' in x else x)
    train['WindSpeed_ob'] = train['WindSpeed_ob'].apply(lambda x: (int(x.split()[0])+int(x.split()[-1]))/2 if not pd.isna(x) and type(x)!=float and 'gusts up to' in x else x)
    train['WindSpeed_dense'] = train['WindSpeed_ob'].apply(strtofloat)
    
     ## Weather
    train['GameWeather_process'] = train['GameWeather'].str.lower() #将字符串转化为小写
    #标准化为统一的表示
    train['GameWeather_process'] = train['GameWeather_process'].apply(lambda x: "indoor" if not pd.isna(x) and "indoor" in x else x)
    train['GameWeather_process'] = train['GameWeather_process'].apply(lambda x: x.replace('coudy', 'cloudy').replace('clouidy', 'cloudy').replace('party', 'partly') if not pd.isna(x) else x)
    train['GameWeather_process'] = train['GameWeather_process'].apply(lambda x: x.replace('clear and sunny', 'sunny and clear') if not pd.isna(x) else x)
    train['GameWeather_process'] = train['GameWeather_process'].apply(lambda x: x.replace('skies', '').replace("mostly", "").strip() if not pd.isna(x) else x)
    #将天气转化为数字表示
    train['GameWeather_dense'] = train['GameWeather_process'].apply(map_weather)
    
    ## Rusher
    #NflId玩家的唯一标识符，NflIdRusher进攻玩家的NflId
    # #player是否为Rusher
    train['IsRusher'] = (train['NflId'] == train['NflIdRusher'])
    train['IsRusher_ob'] = (train['NflId'] == train['NflIdRusher']).astype("object")
    #Rusher所属的Team
    temp = train[train["IsRusher"]][["Team", "PlayId"]].rename(columns={"Team":"RusherTeam"})
    train = train.merge(temp, on = "PlayId")
    #Rusher所属的Team和Player所属的Team是否一致
    train["IsRusherTeam"] = train["Team"] == train["RusherTeam"]
    
    ## dense -> categorical
    #游戏场次，有1，2，3，4，5，5表示overtime
    train["Quarter_ob"] = train["Quarter"].astype("object")
    #结束的场次
    train["Down_ob"] = train["Down"].astype("object")
    #球衣号码
    train["JerseyNumber_ob"] = train["JerseyNumber"].astype("object")
    #混战线的场地线
    train["YardLine_ob"] = train["YardLine"].astype("object")
    # train["DefendersInTheBox_ob"] = train["DefendersInTheBox"].astype("object")
    # train["Week_ob"] = train["Week"].astype("object")
    # train["TimeDelta_ob"] = train["TimeDelta"].astype("object")

    ## Orientation and Dir
    #玩家方位(角度)
    train["Orientation_ob"] = train["Orientation"].apply(lambda x : orientation_to_cat(x)).astype("object")
    #玩家运动方位(角度)
    train["Dir_ob"] = train["Dir"].apply(lambda x : orientation_to_cat(x)).astype("object")
    #将Orientation and Dir方向用三角函数表示
    train["Orientation_sin"] = train["Orientation"].apply(lambda x : np.sin(x/360 * 2 * np.pi))
    train["Orientation_cos"] = train["Orientation"].apply(lambda x : np.cos(x/360 * 2 * np.pi))
    train["Dir_sin"] = train["Dir"].apply(lambda x : np.sin(x/360 * 2 * np.pi))
    train["Dir_cos"] = train["Dir"].apply(lambda x : np.cos(x/360 * 2 * np.pi))
    
    ## diff Score
    #HomeScoreBeforePlay主队比赛开始前的得分，VisitorScoreBeforePlay客队在比赛前的得分
    train["diffScoreBeforePlay"] = train["HomeScoreBeforePlay"] - train["VisitorScoreBeforePlay"]
    train["diffScoreBeforePlay_binary_ob"] = (train["HomeScoreBeforePlay"] > train["VisitorScoreBeforePlay"]).astype("object")
    
    ## Turf场面描述
    Turf = {'Field Turf':'Artificial', 'A-Turf Titan':'Artificial', 'Grass':'Natural', 'UBU Sports Speed S5-M':'Artificial', 
            'Artificial':'Artificial', 'DD GrassMaster':'Artificial', 'Natural Grass':'Natural',
            'UBU Speed Series-S5-M':'Artificial', 'FieldTurf':'Artificial', 'FieldTurf 360':'Artificial',
            'Natural grass':'Natural', 'grass':'Natural', 'Natural':'Natural', 'Artifical':'Artificial',
            'FieldTurf360':'Artificial', 'Naturall Grass':'Natural', 'Field turf':'Artificial', 'SISGrass':'Artificial',
            'Twenty-Four/Seven Turf':'Artificial', 'natural grass':'Natural'} 
    train['Turf'] = train['Turf'].map(Turf)
    
    ## OffensePersonnel进攻性球队位置分组
    temp = train["OffensePersonnel"].iloc[np.arange(0, len(train), 22)].apply(lambda x : pd.Series(OffensePersonnelSplit(x)))
    temp.columns = ["Offense" + c for c in temp.columns]
    temp["PlayId"] = train["PlayId"].iloc[np.arange(0, len(train), 22)]
    train = train.merge(temp, on = "PlayId")

    ## DefensePersonnel防卫性球队位置分组
    temp = train["DefensePersonnel"].iloc[np.arange(0, len(train), 22)].apply(lambda x : pd.Series(DefensePersonnelSplit(x)))
    temp.columns = ["Defense" + c for c in temp.columns]
    temp["PlayId"] = train["PlayId"].iloc[np.arange(0, len(train), 22)]
    train = train.merge(temp, on = "PlayId")


In [7]:
#创建新特征
def create_features(df, deploy=False):
    
    def new_X(x_coordinate, play_direction):
        if play_direction == 'left':
            return 120.0 - x_coordinate
        else:
            return x_coordinate
        
    def new_line(rush_team, field_position, yardline):
        if rush_team == field_position:
            # offense starting at X = 0 plus the 10 yard endzone plus the line of scrimmage
            return yardline+10
        else:
            # half the field plus the yards between midfield and the line of scrimmage
            return 50.0 + (60 - yardline)
    
    #定义新方向，表示在0-360之间    
    def new_orientation(angle, play_direction):
        if play_direction == 'left':
            new_angle = 360.0 - angle
            if new_angle == 360.0:
                new_angle = 0.0
            return new_angle
        else:
            return angle
    
    def defense_centroid(position):
        if position=="MLB"or"ILB":
            return 1
        else:
            return 0
    
    def new_vertical_speed(angel,speed):
        return  speed*np.cos(angel/360 * 2 * np.pi)
    
    def new_horizontal_speed(angel,speed):
        return  speed*np.sin(angel/360 * 2 * np.pi)
    
    
    #定义欧几里得距离
    def euclidean_distance(x1,y1,x2,y2):
        x_diff = (x1-x2)**2
        y_diff = (y1-y2)**2

        return np.sqrt(x_diff + y_diff)
    
    def defense_time(distance,speed):
        return distance/speed
    
     
    def back_direction(orientation):
        if orientation > 180.0:
            return 1
        else:
            return 0
      
    def fun(x):
        if x<=100:
            return x
        else:
            return 100
        
    def update_yardline(df):
        #提取NflId和NflIdRusher相等的行
        new_yardline = df[df['NflId'] == df['NflIdRusher']]
        #根据new_line计算新的yardline
        new_yardline['YardLine'] = new_yardline[['PossessionTeam','FieldPosition','YardLine']].apply(lambda x:
                                                                        new_line(x[0],x[1],x[2]), axis=1)
        new_yardline = new_yardline[['GameId','PlayId','YardLine']]
        return new_yardline
    
    def update_orientation(df, yardline):
        #根据X(球员沿场地长轴的位置)和PlayDirection利用new_X计算新的X
        df['X']= df[['X','PlayDirection']].apply(lambda x: new_X(x[0],x[1]), axis=1)
#         df["Y"]= df[["Y","PlayDirection"]].apply(lambda y: new_Y(y[0],y[1]), axis=1)
        #根据Orientation(玩家的方位)和PlayDirection利用new_orientation计算新的Orientation
        df['Orientation'] = df[['Orientation','PlayDirection']].apply(lambda x: new_orientation(x[0],x[1]), axis=1)
        #根据Dir(玩家运动的角度)和PlayDirection利用new_orientation计算新的Dir
        df['Dir'] = df[['Dir','PlayDirection']].apply(lambda x: new_orientation(x[0],x[1]), axis=1)
        df = df.drop('YardLine', axis=1)
        df = pd.merge(df, yardline, on=['GameId','PlayId'], how='inner')
        return df
    
    #计算最后的特征，也就是一个对中Rusher对应的特征
    def back_features(df):
        #提取NflId和NflIdRusher相等的列
        carriers = df[df['NflId'] == df['NflIdRusher']][['GameId','PlayId','NflIdRusher','X','Y','Orientation','Dir','YardLine']]
        carriers['back_from_scrimmage'] = carriers['YardLine'] - carriers['X']
        carriers['back_oriented_down_field'] = carriers['Orientation'].apply(lambda x: back_direction(x))
        carriers['back_moving_down_field'] = carriers['Dir'].apply(lambda x: back_direction(x))
        carriers = carriers.rename(columns={'X':'back_X',
                                            'Y':'back_Y'})
        carriers = carriers[['GameId','PlayId','NflIdRusher','back_X','back_Y','back_from_scrimmage','back_oriented_down_field','back_moving_down_field']]
        return carriers
    
    def features_relative_to_back(df, carriers):
        player_distance = df[['GameId','PlayId','NflId','X','Y']]
        #列特征的融合，每一个对的back_features一样
        player_distance = pd.merge(player_distance, carriers, on=['GameId','PlayId'], how='inner')
        #提取NflId和NflIdRusher不相等的行
        player_distance = player_distance[player_distance['NflId'] != player_distance['NflIdRusher']]
        #计算不相等列的欧几里得距离
        player_distance['dist_to_back'] = player_distance[['X','Y','back_X','back_Y']].apply(lambda x: 
                                                        euclidean_distance(x[0],x[1],x[2],x[3]), axis=1)
        player_distance = player_distance.groupby(['GameId','PlayId','back_from_scrimmage'])\
                                         .agg({'dist_to_back':["min",'max','mean','std']})\
                                         .reset_index()
        player_distance.columns = ['GameId','PlayId','back_from_scrimmage','min_dist','max_dist','mean_dist','std_dist']

        return player_distance   
  
    def offense_features(df):
        rusher = df[df['NflId'] == df['NflIdRusher']][['GameId','PlayId','Team','X','Y']]
        rusher.columns = ['GameId','PlayId','RusherTeam','RusherX','RusherY']

        offense = pd.merge(df,rusher,on=['GameId','PlayId'],how='inner')
        #提取Team与RusherTeam相等的行
        offense = offense[offense['Team'] == offense['RusherTeam']][['GameId','PlayId','X','Y','RusherX','RusherY',"S",
                                                                     "A","Dir","Dis","Orientation","YardLine"]]
        #计算相等行的欧几里得距离
        offense['off_dist_to_back'] = offense[['X','Y','RusherX','RusherY']].apply(lambda x: euclidean_distance(x[0],x[1],x[2],x[3]), axis=1)
#         offense['off_time']=offense[["off_dist_to_back","S"]].apply(lambda x: defense_time(x[0],x[1]),axis=1)
        offense["off_back_from_scrimmage"]=offense["YardLine"]-offense["X"]
        offense = offense.groupby(['GameId','PlayId'])\
                             .agg({'off_dist_to_back':['max','mean',"std"],"S":['max','mean'],"Y":["max","min"],"X":["std"],
                                   "A":["mean"],"Dir":["mean"],"Dis":["mean"],"Orientation":['mean'],"off_back_from_scrimmage":["mean"]}).reset_index()
        offense.columns = ['GameId','PlayId','off_max_dist','off_mean_dist',"off_std_dist","off_max_speed","off_mean_speed",
                           "off_max_Y","off_min_Y","off_x_std","off_mean_A","off_mean_Dir","off_mean_Dis","off_mean_Orientation","off_back_from_scrimmage"] 
        offense["off_x_spread"]=offense["off_max_Y"]-offense["off_min_Y"]
        offense.drop(["off_max_Y","off_min_Y"], axis=1, inplace=True)
        return offense

    def defense_features(df):
        rusher = df[df['NflId'] == df['NflIdRusher']][['GameId','PlayId','Team','X','Y']]
        rusher.columns = ['GameId','PlayId','RusherTeam','RusherX','RusherY']

        defense = pd.merge(df,rusher,on=['GameId','PlayId'],how='inner')
        #提取Team与RusherTeam不相等的行
        defense = defense[defense['Team'] != defense['RusherTeam']][['GameId','PlayId','X','Y','RusherX','RusherY',"S","A","Dir","Dis",'Orientation',"YardLine"]]
        #计算不相等列的欧几里得距离
        defense['def_dist_to_back'] = defense[['X','Y','RusherX','RusherY']].apply(lambda x: euclidean_distance(x[0],x[1],x[2],x[3]), axis=1)
        defense["def_back_from_scrimmage"]=defense["YardLine"]-defense["X"]
        defense['def_time']=defense[["def_dist_to_back","S"]].apply(lambda x: defense_time(x[0],x[1]),axis=1)
        defense = defense.groupby(['GameId','PlayId'])\
                         .agg({'def_dist_to_back':['min','max','mean','std'],"S":['max',"mean"],"def_time":['min'],"Y":["max","min"],
                               "X":["std"],"A":["mean"],"Dir":["mean"],"Dis":["mean"],"Orientation":["mean"],"def_back_from_scrimmage":["mean"]}).reset_index()
        defense.columns = ['GameId','PlayId','def_min_dist','def_max_dist','def_mean_dist','def_std_dist',"def_max_S","def_mean_S",
                           "def_min_time","def_max_Y","def_min_Y","def_x_std","def_mean_A","def_mean_Dir","def_mean_Dis","def_mean_Orientation","def_back_from_scrimmage"]
        defense["def_x_spread"]=defense["def_max_Y"]-defense["def_min_Y"]
        defense.drop(["def_max_Y","def_min_Y"], axis=1, inplace=True)
        return defense
    
    
    def def_min_player(df):
        rusher = df[df['NflId'] == df['NflIdRusher']][['GameId','PlayId','X','Y',"S"]]
        rusher.columns = ['GameId','PlayId','RusherX','RusherY',"RusherS"]
        player = pd.merge(df,rusher,on=['GameId','PlayId'],how='inner')
        defense=defense_features(df)
        defense=defense[['GameId','PlayId','def_min_dist']]
        player= pd.merge(player,defense,on=['GameId','PlayId'],how='inner')
        player["dist"]=player[['X','Y','RusherX','RusherY']].apply(lambda x: euclidean_distance(x[0],x[1],x[2],x[3]), axis=1)
        min_player=player[player["dist"]==player["def_min_dist"]][['GameId','PlayId','RusherS',"X","Y",'S',"Dir","Dis","Orientation","YardLine"]]
        min_player["player_vs_rusher_speed"]=min_player[["RusherS","S"]].apply(lambda x :defense_time(x[0],x[1]),axis=1)
        min_player["minplayer_vs_rusher_speed"]=min_player["player_vs_rusher_speed"].apply(lambda x:fun(x))
        min_player["min_player_Dir_sin"]=min_player["Dir"].apply(lambda x : np.sin(x/360 * 2 * np.pi))
        min_player["min_player_Dir_cos"]=min_player["Dir"].apply(lambda x : np.cos(x/360 * 2 * np.pi))
        min_player["min_player_Orientation_sin"]=min_player["Orientation"].apply(lambda x : np.sin(x/360 * 2 * np.pi))
        min_player["min_player_Orientation_cos"]=min_player["Orientation"].apply(lambda x : np.cos(x/360 * 2 * np.pi))
        min_player["min_player_back_from_scrimmage"]=min_player["YardLine"]-min_player["X"]
        min_player=min_player[['GameId','PlayId',"Y","minplayer_vs_rusher_speed","S","Dir","Dis",'Orientation',"min_player_Dir_sin",
                               "min_player_Dir_cos","min_player_Orientation_sin","min_player_Orientation_cos","min_player_back_from_scrimmage"]]
        min_player.columns=['GameId','PlayId',"min_player_Y","minplayer_vs_rusher_speed","min_player_S","min_play_Dir","min_play_Dis",'min_play_Qrientation',
                            "min_player_Dir_sin","min_player_Dir_cos","min_player_Orientation_sin","min_player_Orientation_cos","min_player_back_from_scrimmage"]
        return min_player
    
    def def_mintime_features(df):
        rusher = df[df['NflId'] == df['NflIdRusher']][['GameId','PlayId','X','Y',"S"]]
        rusher.columns = ['GameId','PlayId','RusherX','RusherY',"RusherS"]
        player = pd.merge(df,rusher,on=['GameId','PlayId'],how='inner')
        defense=defense_features(df)
        defense=defense[['GameId','PlayId','def_min_time']]
        player= pd.merge(player,defense,on=['GameId','PlayId'],how='inner')
        player['def_dist_to_back'] = player[['X','Y','RusherX','RusherY']].apply(lambda x: euclidean_distance(x[0],x[1],x[2],x[3]), axis=1)
        player['def_time']=player[["def_dist_to_back","S"]].apply(lambda x: defense_time(x[0],x[1]),axis=1)
        min_time=player[player["def_time"]==player["def_min_time"]][['GameId','PlayId',"X","Y","def_dist_to_back",'S',"Dir","Dis","Orientation","YardLine"]]
        min_time["min_time_Dir_sin"]= min_time["Dir"].apply(lambda x : np.sin(x/360 * 2 * np.pi))
        min_time["min_time_Dir_cos"]= min_time["Dir"].apply(lambda x : np.cos(x/360 * 2 * np.pi))
        min_time["min_time_Orientation_sin"]=min_time["Orientation"].apply(lambda x : np.sin(x/360 * 2 * np.pi))
        min_time["min_time_Orientation_cos"]=min_time["Orientation"].apply(lambda x : np.cos(x/360 * 2 * np.pi))
        min_time["min_time_back_from_scrimmage"]=min_time["YardLine"]-min_time["X"]
        min_time= min_time[['GameId','PlayId',"Y","def_dist_to_back","S","Dir","Dis","Orientation","min_time_Dir_sin",
                            "min_time_Dir_cos","min_time_Orientation_sin","min_time_Orientation_cos","min_time_back_from_scrimmage"]]
        min_time.columns=['GameId','PlayId',"min_time_Y","min_time_dist","min_time_S","min_time_Dir","min_time_Dis","min_time_Orientation","min_time_Dir_sin",
                          "min_time_Dir_cos","min_time_Orientation_sin","min_time_Orientation_cos","min_time_back_from_scrimmage"]
        return min_time
      
    def def_max_speed(df):
        rusher = df[df['NflId'] == df['NflIdRusher']][['GameId','PlayId','X','Y',"S"]]
        rusher.columns = ['GameId','PlayId','RusherX','RusherY',"RusherS"]
        player = pd.merge(df,rusher,on=['GameId','PlayId'],how='inner')
        defense=defense_features(df)
        defense=defense[['GameId','PlayId','def_max_S']]
        player= pd.merge(player,defense,on=['GameId','PlayId'],how='inner')
        max_speed=player[player["S"]==player["def_max_S"]][['GameId','PlayId','RusherX','RusherY',"RusherS","X","Y","S"]]
        max_speed=max_speed.groupby(['GameId','PlayId'])\
                         .agg({"RusherX":["mean"],"RusherY":["mean"],"RusherS":["mean"],"X":["mean"],"Y":["mean"],"S":["mean"]}).reset_index()
        max_speed["max_speed_dist"]=max_speed[['X','Y','RusherX','RusherY']].apply(lambda x: euclidean_distance(x[0],x[1],x[2],x[3]), axis=1)
        max_speed["max_speed_vs_rusher_S"]=max_speed[["RusherS","S"]].apply(lambda x :defense_time(x[0],x[1]),axis=1)
        max_speed["max_speed_vs_rusher_S"]=max_speed["max_speed_vs_rusher_S"].apply(lambda x:fun(x))
        max_speed["max_speed_time"]=max_speed[["max_speed_dist","S"]].apply(lambda x: defense_time(x[0],x[1]),axis=1)
        max_speed=max_speed[['GameId','PlayId',"max_speed_dist","max_speed_vs_rusher_S","max_speed_time"]]
        return max_speed
    
    def qb_features(df):
        rusher = df[df['NflId'] == df['NflIdRusher']][['GameId','PlayId','X','Y']]
        rusher.columns = ['GameId','PlayId','RusherX','RusherY']
        qb= pd.merge(df,rusher,on=['GameId','PlayId'],how='inner')
        qb_feats=qb[qb["Position"]=="QB"][["GameId","PlayId","X","Y","RusherX","RusherY","S"]]
        qb_feats["distance_to_qb"]=qb[['X','Y','RusherX','RusherY']].apply(lambda x: euclidean_distance(x[0],x[1],x[2],x[3]), axis=1)
        qb_feats["time_to_qb"]=qb_feats[["distance_to_qb","S"]].apply(lambda x: defense_time(x[0],x[1]),axis=1)
        qb_feats["time_to_qb"]=qb_feats["time_to_qb"].apply(lambda x: fun(x))
        qb_feats=qb_feats[["GameId","PlayId","distance_to_qb","time_to_qb","S","Y"]] 
        qb_feats=qb_feats.groupby(['GameId','PlayId'])\
                         .agg({"distance_to_qb":['mean'],'time_to_qb':['mean'],"S":["mean"],"Y":["mean"]})\
                         .reset_index()
        qb_feats.columns=["GameId","PlayId","distance_to_qb","time_to_qb","qb_S","qb_Y"]
        return qb_feats
    
     #定义静态特征
    def static_features(df):
        add_new_feas = []
        ## Height
        df['PlayerHeight_dense'] = df['PlayerHeight'].apply(lambda x: 12*int(x.split('-')[0])+int(x.split('-')[1]))
        add_new_feas.append('PlayerHeight_dense')

        ## Time
        df['TimeHandoff'] = df['TimeHandoff'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"))
        df['TimeSnap'] = df['TimeSnap'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"))

        df['TimeDelta'] = df.apply(lambda row: (row['TimeHandoff'] - row['TimeSnap']).total_seconds(), axis=1)
        df['PlayerBirthDate'] =df['PlayerBirthDate'].apply(lambda x: datetime.datetime.strptime(x, "%m/%d/%Y"))

        ## Age
        seconds_in_year = 60*60*24*365.25
        df['PlayerAge'] = df.apply(lambda row: (row['TimeHandoff']-row['PlayerBirthDate']).total_seconds()/seconds_in_year, axis=1)
        add_new_feas.append('PlayerAge')

        ## WindSpeed
        df['WindSpeed_ob'] = df['WindSpeed'].apply(lambda x: str(x).lower().replace('mph', '').strip() if not pd.isna(x) else x)
        df['WindSpeed_ob'] = df['WindSpeed_ob'].apply(lambda x: (int(x.split('-')[0])+int(x.split('-')[1]))/2 if not pd.isna(x) and '-' in x else x)
        df['WindSpeed_ob'] = df['WindSpeed_ob'].apply(lambda x: (int(x.split()[0])+int(x.split()[-1]))/2 if not pd.isna(x) and type(x)!=float and 'gusts up to' in x else x)
        df['WindSpeed_dense'] = df['WindSpeed_ob'].apply(strtofloat)
        add_new_feas.append('WindSpeed_dense')

#         ##force
#         df["Force"]=df[["PlayerWeight","A"]].apply(lambda x: force(x[0],x[1]), axis=1)
#         df["Impulse"]=df[["PlayerWeight","S"]].apply(lambda x: force(x[0],x[1]), axis=1)
#         add_new_feas.append('Force')
#         add_new_feas.append("Impulse")
        
        ## Weather
        df['GameWeather_process'] = df['GameWeather'].str.lower()
        df['GameWeather_process'] = df['GameWeather_process'].apply(lambda x: "indoor" if not pd.isna(x) and "indoor" in x else x)
        df['GameWeather_process'] = df['GameWeather_process'].apply(lambda x: x.replace('coudy', 'cloudy').replace('clouidy', 'cloudy').replace('party', 'partly') if not pd.isna(x) else x)
        df['GameWeather_process'] = df['GameWeather_process'].apply(lambda x: x.replace('clear and sunny', 'sunny and clear') if not pd.isna(x) else x)
        df['GameWeather_process'] = df['GameWeather_process'].apply(lambda x: x.replace('skies', '').replace("mostly", "").strip() if not pd.isna(x) else x)
        df['GameWeather_dense'] = df['GameWeather_process'].apply(map_weather)
        add_new_feas.append('GameWeather_dense')
        
#                 ## Rusher
#         df['IsRusher'] = (df['NflId'] == df['NflIdRusher'])
#         df['IsRusher_ob'] = (df['NflId'] == df['NflIdRusher']).astype("object")
#         temp = df[df["IsRusher"]][["Team", "PlayId"]].rename(columns={"Team":"RusherTeam"})
#         df = df.merge(temp, on = "PlayId")
#         df["IsRusherTeam"] = df["Team"] == df["RusherTeam"]

#         # dense -> categorical
#         df["Quarter_ob"] = df["Quarter"].astype("object")
#         df["Down_ob"] =df["Down"].astype("object")
#         df["JerseyNumber_ob"] = df["JerseyNumber"].astype("object")
#         df["YardLine_ob"] = df["YardLine"].astype("object")
#         df["DefendersInTheBox_ob"] = df["DefendersInTheBox"].astype("object")
#         df["Week_ob"] = df["Week"].astype("object")
#         df["TimeDelta_ob"] = df["TimeDelta"].astype("object")


        ## Orientation and Dir
        df["Orientation_ob"] = df["Orientation"].apply(lambda x : orientation_to_cat(x)).astype("object")
        df["Dir_ob"] = df["Dir"].apply(lambda x : orientation_to_cat(x)).astype("object")

        df["Orientation_sin"] = df["Orientation"].apply(lambda x : np.sin(x/360 * 2 * np.pi))
        df["v_horizontal"]= df[['Orientation_sin','S']].apply(lambda x: new_horizontal_speed(x[0],x[1]), axis=1)
        df["Orientation_cos"] = df["Orientation"].apply(lambda x : np.cos(x/360 * 2 * np.pi))
        df["v_vertical"]=df[['Orientation_cos','S']].apply(lambda x: new_vertical_speed(x[0],x[1]), axis=1)
        df["Dir_sin"] = df["Dir"].apply(lambda x : np.sin(x/360 * 2 * np.pi))
        df["Dir_cos"] = df["Dir"].apply(lambda x : np.cos(x/360 * 2 * np.pi))
        add_new_feas.append("Orientation_sin")
        add_new_feas.append("Orientation_cos")
        add_new_feas.append("Dir_sin")
        add_new_feas.append("Dir_cos")
        add_new_feas.append("v_vertical")
        add_new_feas.append("v_horizontal")
        

        ## diff Score
        df["diffScoreBeforePlay"] = df["HomeScoreBeforePlay"] - df["VisitorScoreBeforePlay"]
        add_new_feas.append("diffScoreBeforePlay")
        
        static_features = df[df['NflId'] == df['NflIdRusher']][add_new_feas+['GameId','PlayId','X','Y','S','A','Dis','Orientation','Dir',
                                                            'YardLine','Distance',"Quarter","Down",'DefendersInTheBox',]].drop_duplicates()
        static_features['DefendersInTheBox'] = static_features['DefendersInTheBox'].fillna(np.mean(static_features['DefendersInTheBox']))
        static_features.fillna(-999,inplace=True)
        return static_features
    
    
    
    #根据GameId.PlayId进行特征融合
    def combine_features(relative_to_back,defense,offense,def_min_player,min_time,max_speed,qb_feats,static,deploy=deploy):
        df = pd.merge(relative_to_back,defense,on=['GameId','PlayId'],how='inner')
        df = pd.merge(df,offense,on=['GameId','PlayId'],how='inner')
        df = pd.merge(df,def_min_player,on=['GameId','PlayId'],how='inner')
        df = pd.merge(df,min_time,on=['GameId','PlayId'],how='inner')
        df = pd.merge(df,max_speed,on=['GameId','PlayId'],how='inner')
        df = pd.merge(df,qb_feats,on=['GameId','PlayId'],how='inner')
        df = pd.merge(df,static,on=['GameId','PlayId'],how='inner')
        if not deploy:
            df = pd.merge(df, outcomes, on=['GameId','PlayId'], how='inner')

        return df
    
    yardline = update_yardline(df)
    df = update_orientation(df, yardline)
    back_feats = back_features(df)
    rel_back = features_relative_to_back(df, back_feats)
    def_feats = defense_features(df)
    off_feats = offense_features(df)
    min_def=def_min_player(df)
    min_time_feats=def_mintime_features(df)
    max_speed_feats=def_max_speed(df)
    qb_feats=qb_features(df)
    static_feats = static_features(df)
    basetable = combine_features(rel_back, def_feats,off_feats,min_def,min_time_feats,max_speed_feats,qb_feats,static_feats,deploy=deploy)
    return basetable

In [8]:
%time train_basetable = create_features(train, False)

CPU times: user 12min 26s, sys: 14.8 s, total: 12min 40s
Wall time: 12min 39s


In [9]:
train_basetable.shape

(23170, 86)

In [10]:
train_basetable.head(10)

,GameId,PlayId,back_from_scrimmage,min_dist,max_dist,mean_dist,std_dist,def_min_dist,def_max_dist,def_mean_dist,def_std_dist,def_max_S,def_mean_S,def_min_time,def_x_std,def_mean_A,def_mean_Dir,def_mean_Dis,def_mean_Orientation,def_back_from_scrimmage,def_x_spread,off_max_dist,off_mean_dist,off_std_dist,off_max_speed,off_mean_speed,off_x_std,off_mean_A,off_mean_Dir,off_mean_Dis,off_mean_Orientation,off_back_from_scrimmage,off_x_spread,min_player_Y,minplayer_vs_rusher_speed,min_player_S,min_play_Dir,min_play_Dis,min_play_Qrientation,min_player_Dir_sin,min_player_Dir_cos,min_player_Orientation_sin,min_player_Orientation_cos,min_player_back_from_scrimmage,min_time_Y,min_time_dist,min_time_S,min_time_Dir,min_time_Dis,min_time_Orientation,min_time_Dir_sin,min_time_Dir_cos,min_time_Orientation_sin,min_time_Orientation_cos,min_time_back_from_scrimmage,"(max_speed_dist, )","(max_speed_vs_rusher_S, )","(max_speed_time, )",distance_to_qb,time_to_qb,qb_S,qb_Y,PlayerHeight_dense,PlayerAge,WindSpeed_dense,GameWeather_dense,Orientation_sin,Orientation_cos,Dir_sin,Dir_cos,v_vertical,v_horizontal,diffScoreBeforePlay,X,Y,S,A,Dis,Orientation,Dir,YardLine,Distance,Quarter,Down,DefendersInTheBox,Yards
0,2017090700,20170907000118,3.75,1.449724,22.415872,8.046559,4.873845,4.593310,22.415872,9.752491,5.327299,4.55,1.314545,2.176268,5.294079,1.025455,173.046364,0.184545,156.195455,-3.179091,22.19,13.296834,5.609122,3.983326,5.15,2.106364,1.772665,1.358182,131.217273,0.305455,186.260000,0.942727,19.35,32.64,8.642857,0.42,161.30,0.01,332.39,0.320613,-0.947210,-0.463451,0.886123,-0.33,38.83,9.902020,4.55,169.16,0.51,284.53,0.188067,-0.982156,-0.968016,0.250887,-1.65,9.902020,0.797802,2.176268,1.449724,1.725862,0.84,29.49,70,25.596251,8.0,1.0,-0.309349,-0.950949,0.911690,-0.410878,3.629500,-0.019599,0,41.25,30.53,3.63,3.35,0.38,198.02,114.26,45.0,2,1,3,6.0,8
1,2017090700,20170907000139,4.07,0.792023,23.025872,8.614623,5.598683,4.287773,23.025872,10.297028,5.833217,2.79,1.639091,3.573144,5.406292,1.592727,207.282727,0.197273,165.874545,-3.561818,24.45,18.102928,6.149070,5.126878,3.61,2.094545,2.001380,1.541818,125.943636,0.275455,183.202727,1.132727,22.10,27.55,2.550000,1.20,334.74,0.13,353.12,-0.426727,0.904381,-0.119790,0.992799,-0.20,27.55,4.287773,1.20,334.74,0.13,353.12,-0.426727,0.904381,-0.119790,0.992799,-0.20,10.579986,1.096774,3.792110,0.792023,0.613971,1.29,27.49,70,25.596252,8.0,1.0,0.510543,-0.859852,0.740805,0.671721,3.059655,0.027266,0,48.93,27.16,3.06,2.41,0.34,149.30,47.80,53.0,10,1,1,6.0,3
2,2017090700,20170907000189,3.66,1.646390,20.726285,8.482583,4.642121,4.221670,20.726285,9.903689,5.073290,4.35,3.244545,1.716126,4.720893,2.092727,164.201818,0.360909,247.320000,-3.389091,23.81,13.606149,6.290334,4.128883,5.77,3.682727,2.115286,1.419091,150.955455,0.400000,224.203636,0.151818,22.40,20.43,2.345528,2.46,188.06,0.30,0.07,-0.140210,-0.990122,0.001222,0.999999,-0.35,20.43,4.221670,2.46,188.06,0.30,0.07,-0.140210,-0.990122,0.001222,0.999999,-0.35,14.379447,1.326437,3.305620,1.646390,0.462469,3.56,20.52,70,25.596253,8.0,1.0,-0.631759,-0.775165,0.668612,-0.743612,5.769472,-0.063620,0,71.34,19.11,5.77,2.42,0.60,219.18,138.04,75.0,10,1,1,7.0,5
3,2017090700,20170907000345,3.53,0.918096,9.791231,5.549379,1.983128,4.528002,9.791231,6.309354,1.834174,3.28,1.555455,2.286889,0.962418,1.293636,146.282727,0.199091,210.851818,-1.310909,15.17,8.065346,4.284914,2.277473,4.45,2.141818,1.898686,0.880909,92.389091,0.254545,175.651818,0.280909,11.80,22.64,4.045455,1.10,250.74,0.28,24.47,-0.944031,-0.329855,0.414217,0.910178,-0.09,22.05,4.688123,2.05,327.50,0.25,21.46,-0.537300,0.843391,0.365852,0.930673,0.21,9.037367,1.356707,2.755295,0.918096,0.312278,2.94,24.86,71,26.852933,8.0,1.0,0.108346,-0.994113,0.995496,0.094803,4.449330,0.008415,0,104.47,25.36,4.45,3.20,0.46,173.78,84.56,108.0,2,1,2,9.0,2
4,2017090700,20170907000395,5.01,0.502892,21.214806,9.168819,5.611232,4.288088,21.214806,11.056456,5.900009,3.33,2.322727,1.624276,5.391251,2.121818,198.156364,0.245

In [11]:
# X = train_basetable.copy()
# yards = X.Yards

# y = np.zeros((yards.shape[0], 199))
# for idx, target in enumerate(list(yards)):
#     y[idx][99 + target] = 1

# X.drop(['GameId','PlayId','Yards'], axis=1, inplace=True)
# X_train=X
# y_train= yards

In [12]:
# import xgboost as xgb
# from xgboost.sklearn import XGBClassifier
# xgb_clf=XGBClassifier(max_depth=4,num_class=199,n_estimators=100,learning_rate=0.1,gamma=0.11,colsample_bylevel=0.6)
# xgb_clf.fit(X_train,y_train)

In [13]:
# import matplotlib.pyplot as plt
# fig,ax = plt.subplots(figsize=(20,20))
# xgb.plot_importance(xgb_clf,grid=False,ax=ax,importance_type="weight")
# plt.show()

In [14]:
X = train_basetable.copy()
yards = X.Yards

y = np.zeros((yards.shape[0], 199))
for idx, target in enumerate(list(yards)):
    y[idx][99 + target] = 1

X.drop(['GameId','PlayId','Yards'], axis=1, inplace=True)

In [15]:
# from sklearn.decomposition import PCA
# from sklearn.pipeline import make_pipeline
scaler=StandardScaler()
X=scaler.fit_transform(X)

In [16]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15, random_state=12345)
print(X_train.shape, X_val.shape)
print(y_train.shape, y_val.shape)

(19694, 83) (3476, 83)
(19694, 199) (3476, 199)


In [17]:
from keras.layers import Dense,Input,Flatten,concatenate,Dropout,Lambda
from keras.models import Model
import keras.backend as K
import re
from keras.losses import binary_crossentropy
from  keras.callbacks import EarlyStopping,ModelCheckpoint
import codecs

from keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
from sklearn.metrics import f1_score




class CRPSCallback(Callback):
    
    def __init__(self,validation, predict_batch_size=20, include_on_batch=False):
        super(CRPSCallback, self).__init__()
        self.validation = validation
        self.predict_batch_size = predict_batch_size
        self.include_on_batch = include_on_batch
        
        print('validation shape',len(self.validation))

    def on_batch_begin(self, batch, logs={}):
        pass

    def on_train_begin(self, logs={}):
        if not ('CRPS_score_val' in self.params['metrics']):
            self.params['metrics'].append('CRPS_score_val')

    def on_batch_end(self, batch, logs={}):
        if (self.include_on_batch):
            logs['CRPS_score_val'] = float('-inf')

    def on_epoch_end(self, epoch, logs={}):
        logs['CRPS_score_val'] = float('-inf')
            
        if (self.validation):
            X_valid, y_valid = self.validation[0], self.validation[1]
            y_pred = self.model.predict(X_valid)
            y_true = np.clip(np.cumsum(y_valid, axis=1), 0, 1)
            y_pred = np.clip(np.cumsum(y_pred, axis=1), 0, 1)
            val_s = ((y_true - y_pred) ** 2).sum(axis=1).sum(axis=0) / (199 * X_valid.shape[0])
            val_s = np.round(val_s, 6)
            logs['CRPS_score_val'] = val_s
    

In [18]:
def get_model(x_tr,y_tr,x_val,y_val):
    inp = Input(shape = (x_tr.shape[1],))
    x = Dense(400, input_dim=X.shape[1], activation='elu')(inp)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(128, activation='elu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(64, activation='elu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    
    
    
    out = Dense(120, activation='softmax')(x)
    model = Model(inp,out)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[])
    #add lookahead
#     lookahead = Lookahead(k=5, alpha=0.5) # Initialize Lookahead
#     lookahead.inject(model) # add into model

    
    es = EarlyStopping(monitor='CRPS_score_val', 
                       mode='min',
                       restore_best_weights=True, 
                       verbose=1, 
                       patience=20)

    mc = ModelCheckpoint('best_model.h5',monitor='CRPS_score_val',mode='min',
                                   save_best_only=True, verbose=1, save_weights_only=True)
    
    bsz = 256
    steps = x_tr.shape[0]/bsz
    


    model.fit(x_tr, y_tr,callbacks=[CRPSCallback(validation = (x_val,y_val)),es,mc], epochs=200, batch_size=bsz,verbose=1)
    
    y_pred = model.predict(x_val)
    y_valid = y_val
    y_true = np.clip(np.cumsum(y_valid, axis=1), 0, 1)
    y_pred = np.clip(np.cumsum(y_pred, axis=1), 0, 1)
    val_s = ((y_true - y_pred) ** 2).sum(axis=1).sum(axis=0) / (199* x_val.shape[0])
    crps = np.round(val_s, 6)

    return model,crps

In [19]:
from sklearn.model_selection import train_test_split, KFold
import time

losses = []
models = []
crps_csv = []

s_time = time.time()


for k in range(2):
    kfold = KFold(5, random_state = 42 + k, shuffle = True)
    for k_fold, (tr_inds, val_inds) in enumerate(kfold.split(yards)):
        print("-----------")
        print("-----------")
        tr_x,tr_y = X[tr_inds],y[tr_inds]
        val_x,val_y = X[val_inds],y[val_inds]
        tr_y,val_y = tr_y[:,79:],val_y[:,79:]
        model,crps = get_model(tr_x,tr_y,val_x,val_y)
        models.append(model)
        print("the %d fold crps is %f"%((k_fold+1),crps))
        crps_csv.append(crps)
print("mean crps is %f"%np.mean(crps_csv))


def predict(x_te):
    model_num = len(models)
    for k,m in enumerate(models):
        if k==0:
            y_pred = m.predict(x_te,batch_size=1024)
        else:
            y_pred+=m.predict(x_te,batch_size=1024)
            
    y_pred = y_pred / model_num
    
    return y_pred
            

-----------
-----------
validation shape 2
Epoch 1/200
18536/18536 [==============================] - 2s 110us/step - loss: 4.9635

Epoch 00001: CRPS_score_val improved from inf to 0.07907, saving model to best_model.h5
Epoch 2/200
18536/18536 [==============================] - 1s 57us/step - loss: 4.3482

Epoch 00002: CRPS_score_val improved from 0.07907 to 0.05306, saving model to best_model.h5
Epoch 3/200
18536/18536 [==============================] - 1s 56us/step - loss: 3.7815

Epoch 00003: CRPS_score_val improved from 0.05306 to 0.02769, saving model to best_model.h5
Epoch 4/200
18536/18536 [==============================] - 1s 56us/step - loss: 3.2883

Epoch 00004: CRPS_score_val improved from 0.02769 to 0.01662, saving model to best_model.h5
Epoch 5/200
18536/18536 [==============================] - 1s 56us/step - loss: 3.0112

Epoch 00005: CRPS_score_val improved from 0.01662 to 0.01399, saving model to best_model.h5
Epoch 6/200
18536/18536 [==============================] - 1

In [20]:
print("mean crps is %f"%np.mean(crps_csv))

mean crps is 0.012406


In [21]:
%time
if  TRAIN_OFFLINE==False:
    from kaggle.competitions import nflrush
    env = nflrush.make_env()
    iter_test = env.iter_test()

    for (test_df, sample_prediction_df) in iter_test:
        basetable = create_features(test_df, deploy=True)
        basetable.drop(['GameId','PlayId'], axis=1, inplace=True)
        scaled_basetable = scaler.transform(basetable)
        y_pred = predict(scaled_basetable)
        y_pred = np.hstack((np.zeros((y_pred.shape[0],79)),y_pred))
        y_pred = np.clip(np.cumsum(y_pred, axis=1), 0, 1).tolist()[0]
        

        preds_df = pd.DataFrame(data=[y_pred], columns=sample_prediction_df.columns)
        env.predict(preds_df)

    env.write_submission_file()

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 54.8 µs
Your submission file has been saved!  Once you `Commit` your Notebook and it finishes running, you can submit the file to the competition from the Notebook Viewer `Output` tab.
